In [1]:
from pyspark import SparkContext,SparkConf
from itertools import combinations
import time
import math
import json
import xgboost as xgb
import sys
import csv

In [2]:
'''
input_file_path = sys.argv[1]
output_file_path = sys.argv[2]
'''
input_folder = "/Users/gopi/Desktop/Assignment3/"
test_file_path = "/Users/gopi/Desktop/Assignment3/yelp_val_in.csv"
output_file_path = "/Users/gopi/Desktop/Assignment3/output_2_3.csv"
output_file_path_pairs = "/Users/gopi/Desktop/Assignment3/output_1_sim_m.csv"
#SparkContext.setSystemProperty('spark.executor.memory', '2g')
conf = SparkConf().setMaster("local[*]").setAppName("assign-3-task-1").set("spark.executor.memory", "4g").set("spark.driver.memory", "4g")
sc = SparkContext(conf=conf)
sc.setLogLevel("ERROR")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/03/21 18:13:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
22/03/21 18:13:23 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
22/03/21 18:13:23 WARN Utils: Service 'SparkUI' could not bind on port 4041. Attempting port 4042.


In [3]:
def filter_pairs(user_ratings_1, user_ratings_2):
    if user_ratings_1 is None or user_ratings_2 is None:
        return False
    #print("set 1 is :", user_ratings_1)
    #print("set 2 is :", user_ratings_2)
    set_1 = set(user_ratings_1.keys())
    set_2 = set(user_ratings_2.keys())
    intersection = list(set_1.intersection(set_2))
    #print("inter is :", intersection)
    #print(len(intersection) > 0)
    return len(intersection) >= 1

def compute_similarity(user_ratings_1, user_ratings_2):
    set_1 = set(user_ratings_1.keys())
    set_2 = set(user_ratings_2.keys())
    corated = list(set_1.intersection(set_2))
    values_set_1 = list(user_ratings_1.values())
    values_set_2 = list(user_ratings_2.values())
    
    list_1 = []
    list_2 = []
    for user_id in corated:
        list_1.append(user_ratings_1[user_id])
        list_2.append(user_ratings_2[user_id])
    
    avg_1 = sum(list_1) / float(len(list_1))
    avg_2 = sum(list_2) / float(len(list_2))
    
    avg_1 = sum(values_set_1) / len(values_set_1)
    avg_2 = sum(values_set_2) / len(values_set_2)   
    
    num = 0
    denom_1 = 0
    denom_2 = 0
    for i in range(len(list_1)):
        num += ((list_1[i] - avg_1) * (list_2[i] - avg_2))
        denom_1 += ((list_1[i] - avg_1) ** 2)
        denom_2 += ((list_2[i] - avg_2) ** 2)
    
    denom = math.sqrt(denom_1) * math.sqrt(denom_2)
    if num == 0 or denom == 0:
        return 0
    
    return num / denom
    
    
def predict_rating(x):
    if x[0] not in user_index_map:
        return avg_business_rating[x[1]] if x[1] in business_index_map else 4.0
    
    if x[1] not in business_index_map:
        return avg_user_rating[x[0]] if x[0] in user_index_map else 4.0
    
    target_user_id = user_index_map[x[0]]
    target_business_id = business_index_map[x[1]]
    user_rated_business = user_business_rating_map[target_user_id]
    #limit = 5
    final = []
    for business in user_rated_business:
        business_id = business[0]
        key = (target_business_id, business_id) if target_business_id < business_id else (business_id, target_business_id)
        similarity = similarity_map[key] if key in similarity_map else 0.1
        #similarity = (similarity ** 2.5)
        if business_id != target_business_id:
            final.append((similarity, business[1]))
    
    #final = sorted(final, key = lambda x : x[0], reverse=True)
    #final = final[:limit]
    #print(final)
    num = 0
    denom = 0
    for pair in final:
        num += (pair[1] * pair[0])
        denom += pair[0]
    if denom == 0:
        return avg_user_rating[x[0]] if x[0] in user_index_map else 4.0
    return num / denom

def get_hybrid_prediction(pred_1, pred_2, X_index):
    final_pred = []
    for i in range(len(pred_1)):
        data = [pred_1[i][0],pred_1[i][1]]
        count = n_count_rdd[X_index[i][0]]
        
        if count >= 1000 :
            new_pred = (0.9 * pred_1[i][2]) + (0.1 * pred_2[i])
        else:
            new_pred = (0.1 * pred_1[i][2]) + (0.9 * pred_2[i])
        
        data.append(new_pred)
        final_pred.append(data)       
    return final_pred

def compute_rmse(pred_data,data_type):

    file_2 = open("/Users/gopi/Desktop/Assignment3/yelp_val.csv", "r")
    csv_2 = csv.reader(file_2, delimiter=",")
    pred_2 = []
    c = 0
    for row in csv_2:
        if c != 0:
            pred_2.append(float(row[2]))
        c+=1
            
    pred_1 = []
    if data_type == "CF":
        pred_1 = [data[2] for data in pred_data]
    else:
        pred_1 = pred_data
        
    total = 0
    for i in range(len(pred_1)):
        total += ((pred_1[i] - pred_2[i]) ** 2)
    
    return math.sqrt(total/len(pred_1))

def update_predictions(predictions):
    results = []
    for pred in predictions:
        rating = pred if pred <= 5.0 else 5.0
        rating = 3.0 if rating <= 1.5 else rating
        results.append(rating)
    return results

def get_neighbours_count(x):
    if x[0] not in user_index_map:
        return (x[0], 0)
    
    if x[1] not in business_index_map:
        return (x[0],0)
    
    target_user_id = user_index_map[x[0]]
    target_business_id = business_index_map[x[1]]
    user_rated_business = user_business_rating_map[target_user_id]
    #limit = 5
    final = []
    for business in user_rated_business:
        business_id = business[0]
        key = (target_business_id, business_id) if target_business_id < business_id else (business_id, target_business_id)
        similarity = similarity_map[key] if key in similarity_map else 0.1
        #similarity = (similarity ** 2.5)
        if business_id != target_business_id:
            final.append((similarity, business[1]))     
    return  (x[0], len(final))

def write_data_to_file(output_file_path, data):
    file = open(output_file_path, 'w')
    file.write("user_id, business_id, prediction")
    file.write("\n")
    for i in range(len(data)):
        key = data[i]
        rating = key[2] if key[2] < 5.0 else 5.0
        rating = 3.0 if rating < 1.5 else rating
        rating = round(rating, 3)
        output = key[0] + "," + key[1] + "," + str(rating)
        file.write(output)
        file.write("\n")
    file.close()   
        

In [4]:
def filter_users(user_id) :
    return user_id in user_id_set

def filter_business(business_id):
    return business_id in business_id_set

def load_user_data(user):
    result = [user['user_id'], user['review_count'], user['average_stars'], user.get('fans',0), user.get('useful', 0),user.get('cool',0),user.get('funny',0)]
    return tuple(result)

def load_business_data(business):
    result = [business['business_id'], business['review_count'], business['stars'], business.get('latitude', 0), business.get('longitude',0), business_photo_count.get(business['business_id'],0)]
    return tuple(result)

def get_final_data(data):
    time_1 = time.time()
    user_id = data[0]
    business_id = data[1]
    
    final_data = []
    user_id_data = user_data[user_id] if user_id in user_id_set else [0 for i in range(6)]
    business_id_data = business_data[business_id] if business_id in business_id_set else [0 for i in range(5)]
    
    final_data.extend(user_id_data)
    final_data.extend(business_id_data)
    if len(data) == 3:
        final_data.append(data[2])
    return final_data

def update_predictions(predictions):
    results = []
    for pred in predictions:
        rating = pred if pred <= 5.0 else 5.0
        rating = 3.0 if rating <= 1.5 else rating
        results.append(round(rating, 3))
    return results



In [5]:
start = time.time()
initial_rdd = sc.textFile(input_folder + "yelp_train.csv")
first = initial_rdd.first()
initial_rdd = initial_rdd.filter(lambda line : line != first).map(lambda line : line.split(",")).filter(lambda x : x[0] != None and x[1] != None and x[2] != None).map(lambda line: (line[0], line[1], float(line[2])))

user_index_map = initial_rdd.map(lambda x : x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[0],x[1])).collectAsMap()
business_index_map = initial_rdd.map(lambda x : x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[0],x[1])).collectAsMap()

index_user_map = initial_rdd.map(lambda x : x[0]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[1],x[0])).collectAsMap() 
index_business_map = initial_rdd.map(lambda x : x[1]).distinct().sortBy(lambda x: x).zipWithIndex().map(lambda x : (x[1],x[0])).collectAsMap()

test_data_rdd = sc.textFile(test_file_path)
first = test_data_rdd.first()
test_data_rdd = test_data_rdd.filter(lambda x : x != first).map(lambda line : line.split(","))
business_id_set = set(test_data_rdd.map(lambda x : x[1]).collect())

filtered_input = initial_rdd.map(lambda x: (business_index_map[x[1]], (user_index_map[x[0]], x[2]))).groupByKey().mapValues(list).mapValues(lambda x : dict(x))
business_user_rating_map = filtered_input.collectAsMap()

user_business_rating_map = initial_rdd.map(lambda x: (user_index_map[x[0]], (business_index_map[x[1]], x[2]))).groupByKey().mapValues(list).collectAsMap()

business_items = filtered_input.map(lambda x: x[0])

business_pairs = business_items.cartesian(business_items).filter(lambda x : x[0] < x[1]).filter(lambda x : x[0] in business_id_set).filter(lambda x : filter_pairs(business_user_rating_map[x[0]], business_user_rating_map[x[1]])).partitionBy(15)
similarity_map = business_pairs.map(lambda x : (x, compute_similarity(business_user_rating_map[x[0]], business_user_rating_map[x[1]]))).filter(lambda x : x[1] > 0.01).collectAsMap()


avg_business_rating = initial_rdd.map(lambda x : (x[1], float(x[2]))).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1])/len(x[1]))).collectAsMap()
avg_user_rating = initial_rdd.map(lambda x : (x[0], float(x[2]))).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1])/len(x[1]))).collectAsMap()
results_rdd = test_data_rdd.map(lambda x : (x[0], x[1], predict_rating(x))).collect()



/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling
/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with s

In [7]:
#Model based predictions

user_id_set = initial_rdd.map(lambda x : (x[0])).distinct().collect()
business_id_set = initial_rdd.map(lambda x : (x[1])).distinct().collect()

business_photo_count = sc.textFile("photo.json").map(lambda x : json.loads(x)).map(lambda x : (x['business_id'], x['photo_id'], x['caption'])).filter(lambda x : len(x[2]) != 0).map(lambda x : (x[0], x[1])).groupByKey().mapValues(set).map(lambda x : (x[0], len(x[1]))).collectAsMap()
#review_length_avg = sc.textFile("review_train.json").map(lambda x : json.loads(x)).map(lambda x : (x['user_id'], len(x['text']))).groupByKey().mapValues(list).map(lambda x : (x[0], sum(x[1]) / len(x[1]))).collectAsMap()

user_data = sc.textFile("user.json").map(lambda x : json.loads(x)).map(lambda user : load_user_data(user)).filter(lambda x : filter_users(x[0])).map(lambda x : (x[0],(x[1:]))).collectAsMap()
business_data = sc.textFile("business.json").map(lambda x : json.loads(x)).map(lambda user : load_business_data(user)).filter(lambda x : filter_business(x[0])).map(lambda x : (x[0],(x[1:]))).collectAsMap()

start_1 = time.time()
X_rdd = initial_rdd.map(lambda x : get_final_data(x))
#print(X_rdd.take(5))
X = X_rdd.map(lambda x : x[:-1]).collect()
Y = X_rdd.map(lambda x : x[-1]).collect()

print(len(user_data))
print(len(business_data))
'''
for key,value in user_business_rating_map.items():
    X.append(get_final_data(key))
    Y.append(value)
'''
    
print("Time taken to load X,Y is:", time.time() - start_1)
xgboost_model = xgb.XGBRegressor()
xgboost_model.fit(X,Y)

testing_rdd = sc.textFile(test_file_path)
testing_header = testing_rdd.first()
testing_data_map = testing_rdd.filter(lambda x : x!= testing_header).map(lambda line : line.split(",")).map(lambda line: ((line[0], line[1]),"")).collectAsMap()

X_test = []
X_index = []
for key in list(testing_data_map.keys()):
    X_test.append(get_final_data(key))
    X_index.append(key)

predictions = xgboost_model.predict(X_test)



/opt/homebrew/lib/python3.9/site-packages/pyspark/python/lib/pyspark.zip/pyspark/shuffle.py:60: UserWarning: Please install psutil to have better support with spilling


11270
24732
Time taken to load X,Y is: 79.6703188419342


In [8]:
#predictions = update_predictions(predictions)
n_count_rdd = test_data_rdd.map(lambda x : get_neighbours_count(x)).collectAsMap()

In [9]:
final_pred = get_hybrid_prediction(results_rdd, predictions, X_index)
write_data_to_file(output_file_path, final_pred)

In [10]:
print(compute_rmse(results_rdd, "CF"))
print(compute_rmse(predictions, "model"))
print(compute_rmse(final_pred, "CF"))

1.0754449514566038
0.9876144246533534
0.9875920144448801
